In [4]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests
import json
import ipyvuetify as viewtify

my_headers        = {'accept' : 'application/json'}
debug_flag        = True
ls_init_display   = True
rs_init_display   = True
ls_qk_fvalues     = []
rs_qk_fvalues     = []
ls_eqn_comps      = []
rs_eqn_comps      = []
ls_dv_comps       = []
rs_dv_comps       = []
ls_eqn_elems      = []
rs_eqn_elems      = []
ls_eqn_btns       = []
rs_eqn_btns       = []
ls_dv_elems       = []
ls_dv_vb_slabels  = []
ls_dv_vb_dvlabels = []
rs_dv_elems       = []
rs_dv_vb_slabels  = []
rs_dv_vb_dvlabels = []
ls_eqn_eval       = ''
rs_eqn_eval       = ''
ls_eqn_eval_text  = ''
rs_eqn_eval_text  = ''
eqn_opers         = ['None', '+', '-', '*', '/', 'exp0.5', 'exp0.3', 'exp-1', 'exp-2', 'exp2', 'exp3', 'exp4', 'sin', 'cos', 'tan']


# debug: A utility function to display debugging information (because I don't yet know a better way to debug)
#
# ARGUMENTS: variable, but first one will be a boolean flag, called debug_flag, to decide whether to print
#            this statement. There is a global debug flag that is tested in the debug function definition
# ALGORITHM: (1) Test to see if the global debug flag is set to True
#                (a) If so, test to see if the 'local' debug flag (argument 0) is True
#                    (i) If so, for the remaining arguments
#                        (o) Print the argument

def debug(*args):
    if (debug_flag == True):
        if (args[0] == True):
            for arg in args[1:]:
                print(arg)

# Initialize the app title, help Text widget and start_value Text widget

file           = open("qudt_logo-300x110.png", "rb")
image          = file.read()
label_style    = {'description_width' : 'initial'}
titleLabel     = widgets.HTML(value  = '<a href = "qadt_help.html"><b>Dimensional Analysis Tool</b></a>')
title_text     = viewtify.Html(tag   = "div", children = ['Dimensional Analysis Tool'], style_ = " color: #194188; font-family: helvetica; font-size: 20px")
#title_text     = viewtify.Html(tag = "div", children = [titleLabel], style_ = " color: #194188; font-family: helvetica; font-size: 20px") # colors.indigo.darken4  194188
titleBox       = widgets.HBox([widgets.Image(value = image, format = 'png', width = 75, height = 27), title_text])
ls_qk_text     = widgets.Text(value  = '', description = 'Type quantity kind name', style = label_style, disabled = False)
rs_qk_text     = widgets.Text(value  = '', description = 'Type quantity kind name', style = label_style, disabled = False)
lseqnLabel     = widgets.Label(value = 'Left Side of Equation')
rseqnLabel     = widgets.Label(value = 'Right Side of Equation')
evalHTML       = viewtify.Html(tag   = "div", children = [ls_eqn_eval_text], style_ = " color: #194188; font-family: helvetica; font-size: 20px")
evalBox        = widgets.HBox([evalHTML])
ls_qk_text_len = 0
rs_qk_text_len = 0

opers_dd = widgets.Dropdown(
    options     = eqn_opers,
    description = 'Possible operators:',
    style       = label_style,
    disabled    = True,
)


# qk_get_request: Constructs and issues the quantity kind SPARQL query, converts to json, 
#                 and parses the ressulting results and bindings values. The bindings are returned.
#
# ARGUMENTS: qk_text  - the text in the qk_text field
# RETURN:    stubinds - the SPARQL query bindings array
# CALLED BY: update_options_from_qk_text

def qk_get_request(qk_text):
    qk_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> PREFIX fn: <http://www.w3.org/2005/xpath-functions%23> SELECT ?qk ?qkl ?qks WHERE { ?qk a qudt:QuantityKind . ?qk rdfs:label ?qkl . ?qk qudt:symbol ?qks . FILTER (fn:starts-with(?qkl, %22"
    qk_query_end   = "%22)) . }"
    qk_query       = qk_query_start + qk_text + qk_query_end
    qk_response    = requests.get(qk_query, headers = my_headers)
    qk_data        = qk_response.json()
    qk_results     = qk_data['results']
    qk_binds       = qk_results['bindings']

    return qk_binds


# get_qks: Parses the 'qk' value array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qk_value, qk_label, qk_symbol tuples - the quantity kind SPARQL query array
# CALLED BY: statement below

def get_qks(obj):
    qk_tuples = []
    i         = 0

    for item in obj:
        elem_qk = item['qk']
        elem_qk_value = elem_qk['value']
        elem_qkl = item['qkl']
        elem_qkl_value = elem_qkl['value']
        elem_qks = item['qks']
        elem_qks_value = elem_qks['value']
        elem_item = (elem_qk_value, elem_qkl_value, elem_qks_value)
        qk_tuples.append(elem_item)
        i += 1

    debug(False, 'qk_tuples:', qk_tuples)
    return qk_tuples


# get_qk_values: Parses the 'qk' value array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qlabels - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_qk_values(obj):
    qk_values = []
    i         = 0

    for item in obj:
        elem = item['qk']
        qk_values.append(elem['value'])
        i += 1

    return qk_values


# get_qk_labels: Parses the 'qkl' label array from the SPARQL query results.
#
# ARGUMENTS: obj     - a bindings array
# RETURN:    qlabels - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_qk_labels(obj):
    qk_labels = []
    i         = 0

    for item in obj:
        elem = item['qkl']
        qk_labels.append(elem['value'])
        i += 1

    return qk_labels


# get_qk_symbols: Parses the 'qk' symbol array from the SPARQL query results.
#
# ARGUMENTS: obj       - a bindings array
# RETURN:    qksymbols - the quantity kind SPARQL query symbols array
# CALLED BY: statement below

def get_qk_symbols(obj):
    qk_symbols = []
    i          = 0

    for item in obj:
        elem = item['qks']
        qk_symbols.append(elem['value'])
        i += 1

    return qk_symbols


# get_qk_labels_and_symbols: Concatenates quantity kind labels and symbols.
#
# ARGUMENTS: qkls, qkss - the labels and symbols arrays
# RETURN:    qklsss     - the concatenation of labels and symbols array
# CALLED BY: statement below

def get_qk_labels_and_symbols(qks):
    qklsss = []
    i      = 0

    while i < len(qks):
        lelem = (qks[i])[1]
        selem = (qks[i])[2]
        qklsss.append(lelem + " - " + selem)
        i += 1

    debug(False, 'qklsss:', qklsss)
    return qklsss


# get_qk_symbol_and_dv: Concatenates quantity kind symbol and dimension vector. NOT FINISHED YET
#
# ARGUMENTS: qks, qkdv - the quantity kind symbol and dimension vector
# RETURN:    qklsss     - the concatenation of labels and symbols array
# CALLED BY: statement below

def get_qk_symbol_and_dv(qks, qkdv):
    return qks + qkdv

# Initialize the ls_qk_dd and rs_qk_dd content (options) by calling the request and constructing 
# the values, labels, symbols, and concatenated values arrays

ls_qk_binds       = qk_get_request(ls_qk_text.value)
ls_qks            = get_qks(ls_qk_binds)
ls_qk_values      = get_qk_values(ls_qk_binds)
ls_qk_labels      = get_qk_labels(ls_qk_binds)
ls_qk_symbols     = get_qk_symbols(ls_qk_binds)
ls_qk_ls_and_ss   = get_qk_labels_and_symbols(ls_qks)
cur_ls_qks        = ls_qks
ls_qk_values_cur  = ls_qk_values
ls_qk_labels_cur  = ls_qk_labels
ls_qk_symbols_cur = ls_qk_symbols

rs_qk_binds       = qk_get_request(rs_qk_text.value)
rs_qks            = get_qks(rs_qk_binds)
rs_qk_values      = get_qk_values(rs_qk_binds)
rs_qk_labels      = get_qk_labels(rs_qk_binds)
rs_qk_symbols     = get_qk_symbols(rs_qk_binds)
rs_qk_ls_and_ss   = get_qk_labels_and_symbols(rs_qks)
cur_rs_qks        = rs_qks
rs_qk_values_cur  = rs_qk_values
rs_qk_labels_cur  = rs_qk_labels
rs_qk_symbols_cur = rs_qk_symbols

# Construct the ls_qk_dd Dropdown widget

ls_qk_dd = widgets.Dropdown(
    options     = ls_qk_ls_and_ss,
    description = 'Possible quantity kinds:',
    style       = label_style,
    disabled    = False,
)

# Construct the ls_qk_dd Dropdown widget for values. This is not used. It is a place to store values and is a HACK.

ls_qkv_dd = widgets.Dropdown(
    options     = ls_qk_values,
    description = 'Possible quantity kinds values:',
    style       = label_style,
    disabled    = True,
)

# Construct the ls_qk_dd Dropdown widget for values. This is not used. It is a place to store values and is a HACK.

ls_qks_dd = widgets.Dropdown(
    options     = ls_qk_symbols,
    description = 'Possible quantity kinds symbols:',
    style       = label_style,
    disabled    = True,
)

# Construct the ls_qk_button Button widget

ls_qk_button = widgets.Button(
    description = 'Add Quantity',
    disabled    = False,
    icon        = 'check',
    align       = 'right'
)

# Construct the rs_qk_dd Dropdown widget

rs_qk_dd = widgets.Dropdown(
    options     = rs_qk_ls_and_ss,
    description = 'Possible quantity kinds:',
    style       = label_style,
    disabled    = False,
)

# Construct the rs_qk_dd Dropdown widget for values. This is not used. It is a place to store values and is a HACK.

rs_qkv_dd = widgets.Dropdown(
    options     = rs_qk_values,
    description = 'Possible quantity kinds values:',
    style       = label_style,
    disabled    = True,
)

# Construct the rs_qk_dd Dropdown widget for symbols. This is not used. It is a place to store symbols and is a HACK.

rs_qks_dd = widgets.Dropdown(
    options     = rs_qk_symbols,
    description = 'Possible quantity kinds symbols:',
    style       = label_style,
    disabled    = True,
)

# Construct the rs_qk_button Button widget

rs_qk_button = widgets.Button(
    description = 'Add Quantity',
    disabled    = False,
    icon        = 'check',
    align       = 'right'
)


# restrict_ls_qk_labels: Restricts the content of the qk_dd dropdown options in qk_labels
#                        list and the qk_text. Used by both sides of the equation.
#
# CALLED BY: update_ls_qk_options_from_input_text
# CALLS:     [python] startswith, append

def restrict_ls_qk_labels(qks, vlist, llist, slist, txt):
    global cur_ls_qks
    global filtered_ls_qks

    item = qks[0]

    debug(False, 'In restrict_qk_labels - Start', item[1], item[0])

    filtered_ls_qks   = []
    cur_ls_qks        = qks

    for qk_item in qks:
        qk_item_value  = qk_item[0]
        qk_item_label  = qk_item[1]
        qk_item_symbol = qk_item[2]

        if qk_item_label.startswith(txt):
            debug(False, 'qk_item_label', qk_item_label)
            filtered_ls_qks.append(qk_item)
            debug(False, 'filtered_ls_qks:', filtered_ls_qks)
        
    if (len(filtered_ls_qks) > 0):
        debug(False, 'In restrict_ls_qk_labels - End', (filtered_ls_qks[0])[1], (filtered_ls_qks[0])[0])

    return filtered_ls_qks


# restrict_rs_qk_labels: Restricts the content of the qk_dd dropdown options in qk_labels
#                        list and the qk_text. Used by both sides of the equation.
#
# CALLED BY: update_rs_qk_options_from_input_text
# CALLS:     [python] startswith, append

def restrict_rs_qk_labels(qks, vlist, llist, slist, txt):
    global cur_rs_qks
    global filtered_rs_qks

    item = qks[0]

    debug(False, 'In restrict_rs_qk_labels - Start', item[1], item[0])
    
    filtered_rs_qks = []
    cur_rs_qks      = qks

    for qk_item in qks:
        qk_item_value  = qk_item[0]
        qk_item_label  = qk_item[1]
        qk_item_symbol = qk_item[2]

        if qk_item_label.startswith(txt):
            debug(False, 'qk_item_label', qk_item_label)
            filtered_rs_qks.append(qk_item)
            debug(False, 'filtered_rs_qks:', filtered_rs_qks)

    if (len(filtered_rs_qks) > 0):
        debug(False, 'In restrict_rs_qk_labels - End', (filtered_rs_qks[0])[1], (filtered_rs_qks[0])[0])

    return filtered_rs_qks


# update_ls_qk_options_from_input_text: Performs model updates downstream of a changed selection in the quant_dd
#                                       dropdown. That is, it updates the start_unit_dd and target_unit_dd 
#                                       model (options) contents, the conversion multipliers, and the converted 
#                                       value.
#
# ARGUMENTS: text (value of the ls_qk_text widget)
# CALLED BY: on_change_lsqkt
# CALLS:     construct_qk_tuples_from_options
#            restrict_qk_labels
#            get_tuples_components

def update_ls_qk_options_from_input_text():
    global ls_qk_text_len

    debug(False, 'In update_ls_qk_options_from_input_text')

    qk_text_v = ls_qk_text.value if ls_qk_text.value else ""

    if len(qk_text_v) > ls_qk_text_len:
        qks_start        = construct_qk_tuples_from_options(ls_qkv_dd.options, ls_qk_dd.options, ls_qks_dd.options)
        qk_values_start  = ls_qkv_dd.options
        qk_labels_start  = ls_qk_dd.options
        qk_symbols_start = ls_qks_dd.options
    elif len(qk_text_v) > 0:
        qks_start        = cur_ls_qks
        qk_values_start  = ls_qk_values_cur
        qk_labels_start  = ls_qk_labels_cur
        qk_symbols_start = ls_qk_symbols_cur
    else:
        debug(False, 'qk_text_v length is 0')
        qks_start        = ls_qks # get_qk_labels_and_symbols (NOPE!)
        qk_values_start  = ls_qk_values
        qk_labels_start  = ls_qk_labels
        qk_symbols_start = ls_qk_symbols

    ls_qk_text_len = len(qk_text_v)
    result         = restrict_ls_qk_labels(qks_start, qk_values_start, qk_labels_start, qk_symbols_start, qk_text_v)

    if len(result) > 0:
        result_arrays     = get_tuples_components(result)
        ls_qkv_dd.options = result_arrays[0]
        ls_qk_dd.options  = result_arrays[1]
        ls_qks_dd.options = result_arrays[2]
    else:
        qk_text_start    = ls_qk_text.value
        ls_qk_text.value = qk_text_start[0: ls_qk_text_len - 1:1]


# construct_qk_tuples_from_options: Given lists of values, lavels, and symbols, construct a list
#                                   of tuples from their aggregated values
# ARGUMENTS: value_options  - The list of values
#            label_options  - The list of labels
#            symbol_options - The list of symbols
# CALLED BY: update_ls_qk_options_from_input_text

def construct_qk_tuples_from_options(value_options, label_options, symbol_options):
    result = []
    i      = 0
    while (i < len(value_options)):
        rtuple = (value_options[i], label_options[i], symbol_options[i])
        result.append(rtuple)
        i += 1

    return result


# get_tuples_components: Given a list of tuples separate them out into their value, label, and symbol component
#                        lists and return them as a tuple
# ARGUMENTS: tuples - The list of tuples to separate
# CALLED BY: update_ls_qk_options_from_input_text

def get_tuples_components(tuples): # get_qk_labels_and_symbols
    value_options  = []
    label_options  = []
    symbol_options = []

    for tuple in tuples:
        value_options.append(tuple[0])
        label_options.append(tuple[1])
        symbol_options.append(tuple[2])

    return value_options, label_options, symbol_options


# on_change_lsqkt: Handles changes to the quantity kind input text widget
#
# CALLED BY: Changes to ls_qk_text value
# CALLS:     update_qk_options_from_input_text

def on_change_lsqkt(change):
    if change['type'] == 'change' and change['name'] == 'value':
        txt_filter_result = update_ls_qk_options_from_input_text()

ls_qk_text.observe(on_change_lsqkt)


# update_rs_qk_options_from_input_text: Performs model updates downstream of a changed selection in the quant_dd
#                                       dropdown. That is, it updates the start_unit_dd and target_unit_dd 
#                                       model (options) contents, the conversion multipliers, and the converted 
#                                       value.
#
# ARGUMENTS: text (value of the ls_qk_text widget)
# CALLED BY: on_change_lsqkt
# CALLS:     restrict_qk_labels

def update_rs_qk_options_from_input_text():
    debug(False, 'In update_rs_qk_options_from_input_text')
    global rs_qk_text_len
    qk_text_v = rs_qk_text.value if rs_qk_text.value else ""

    if len(qk_text_v) > rs_qk_text_len:
        qks_start        = construct_qk_tuples_from_options(rs_qkv_dd.options, rs_qk_dd.options, rs_qks_dd.options)
        qk_values_start  = rs_qkv_dd.options
        qk_labels_start  = rs_qk_dd.options
        qk_symbols_start = rs_qks_dd.options
    elif len(qk_text_v) > 0:
        qks_start        = cur_rs_qks
        qk_values_start  = rs_qk_values_cur
        qk_labels_start  = rs_qk_labels_cur
        qk_symbols_start = rs_qk_symbols_cur
    else:
        qks_start        = rs_qks
        qk_values_start  = rs_qk_values
        qk_labels_start  = rs_qk_labels
        qk_symbols_start = rs_qk_symbols

    rs_qk_text_len = len(qk_text_v)
    result         = restrict_rs_qk_labels(qks_start, qk_values_start, qk_labels_start, qk_symbols_start, qk_text_v)

    if len(result[0]) > 0:
        result_arrays     = get_tuples_components(result)
        rs_qkv_dd.options = result_arrays[0]
        rs_qk_dd.options  = result_arrays[1]
        rs_qks_dd.options = result_arrays[2]
    else:
        qk_text_start    = rs_qk_text.value
        rs_qk_text.value = qk_text_start[0: rs_qk_text_len - 1:1]

# on_change_rsqkt: Handles changes to the quantity kind input text widget
#
# CALLED BY: Changes to rs_qk_text value
# CALLS:     update_rs_qk_options_from_input_text

def on_change_rsqkt(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_rs_qk_options_from_input_text()

rs_qk_text.observe(on_change_rsqkt)


# update_ls_eqn_from_button: Performs model updates downstream of a left equation side button selection
#                            It updates the left side equation quantity list and gets the selected
#                            quantity kind's dimension vector and adds to the left side equation dimension
#                            vector list.
#
# ALGORITHM: (1) Append the selected quantity kind to the equation elements list
#            (2) Get the associated dimension vector for this quantity kind
#            (3) Construct the symbol/dimension vector string
#            (4) Append the string to the dimension vector list
#
# CALLED BY: on_change_lsqkb
# CALLS:     update_eqn_elem_content
#            eqn_elem_dv_get_request
#            get_ls_eqn_elem_dv_value
#            get_ls_eqn_elem_dv_label
#            construct_qk_symbols_dv_tuple
#            update_ls_eqn_dv_content
#            qk_value_for_elem_symbol
#            eqn_elem_dv_exps_get_request
#            get_eqn_elem_amount_exp_value
#            get_eqn_elem_current_exp_value
#            get_eqn_elem_length_exp_value
#            get_eqn_elem_luminosity_exp_value
#            get_eqn_elem_mass_exp_value
#            get_eqn_elem_temperature_exp_value
#            get_eqn_elem_time_exp_value
#            get_eqn_elem_dimensionless_exp_value
#            construct_eval_dv_from_exps

def update_ls_eqn_from_button(*args): # *args represent zero (case here) or more arguments.
    global ls_eqn_elems
    global ls_init_display
    global ls_dv_elems
    global ls_eqn_eval

    lsevalamtexp      = 0
    lsevalcrtexp      = 0
    lsevallthexp      = 0
    lsevalluminexp    = 0
    lsevalmassexp     = 0
    lsevaltempexp     = 0
    lsevaltimeexp     = 0
    lsevaldlessexp    = 0
    ls_eqn_eval_items = []

    debug(False, 'In update_ls_eqn_from_button')
    debug(False, 'length of ls_eqn_elems before update: ' + str(len(ls_eqn_elems)))
    debug(False, 'ls_eqn_elems BEFORE update: ', ls_eqn_elems)
    debug(False, 'length of ls_dv_elems efore update: ' + str(len(ls_dv_elems)))

    selected_qk_label = ls_qk_dd.options[ls_qk_dd.index]
    selected_elem     = find_elem_tuple_from_option_label(selected_qk_label, ls_qks)
    debug(False, 'selected_elem:', selected_elem)

    if (ls_init_display == False):
        ls_eqn_elems.append(selected_elem)

    update_ls_eqn_elem_content(selected_elem)

    debug(False, 'length of ls_eqn_elems AFTER update: ' + str(len(ls_eqn_elems)), 'ls_eqn_elems after update:', ls_eqn_elems)
    debug(False, 'length of ls_eqn_btns AFTER update: ' + str(len(ls_eqn_btns)), 'ls_eqn_btns after update:', ls_eqn_btns)

    # Get the dimension vector for the selected quantity kind

    lseqndv_binds = eqn_elem_dv_get_request(ls_qkv_dd.options[ls_qk_dd.index])
    lseqndvv      = get_eqn_elem_dv_value(lseqndv_binds)
    debug(False, 'lseqndvv: ', lseqndvv)
    lseqndvl      = get_eqn_elem_dv_label(lseqndv_binds)

    # Construct the quantity kind, label, and symbol tuple

    lseqnqksdv = construct_qk_symbols_dv_tuple(ls_qk_dd.options[ls_qk_dd.index], lseqndvl, ls_qk_dd.options, ls_qks_dd.options)

    debug(False, 'ls_dv_elems BEFORE update:', ls_dv_elems, 'lseqnqksdv:', lseqnqksdv)

    if (ls_init_display == False):
        ls_dv_elems.append(lseqnqksdv)

    update_ls_eqn_dv_content(lseqnqksdv)

    debug(False, 'length of ls_dv_elems AFTER update: ' + str(len(ls_dv_elems)), 'ls_dv_elems AFTER update', ls_dv_elems)

    ls_init_display = False

    # Construct evaluation component content (not really written yet but will have to include these kinds of things)

    if (len(ls_eqn_elems) == 1):
        ls_eqn_eval = lseqnqksdv[1]
    else:
        for eqn_elem in ls_eqn_elems:
            debug(False, 'Construct evaluation component')
            debug(False, 'eqn_elem: ', eqn_elem, 'ls_qk_symbols:', ls_qk_symbols, 'ls_qk_values', ls_qk_values)
#            ls_qk_value     = qk_value_for_eqn_elem(eqn_elem)
            ls_qk_value     = eqn_elem[0]
            debug(False, 'ls_qk_value: ', ls_qk_value)
            lsevalexp_binds = eqn_elem_dv_exps_get_request(ls_qk_value)
            debug(False, 'lsevalexp_binds:', lsevalexp_binds)
            lsevalamtexp   += get_eqn_elem_amount_exp_value(lsevalexp_binds)
            debug(False, 'lsevalamtexp: ' + str(lsevalamtexp))
            lsevalcrtexp   += get_eqn_elem_current_exp_value(lsevalexp_binds)
            debug(False, 'lsevalcrtexp: ' + str(lsevalcrtexp))
            lsevallthexp   += get_eqn_elem_length_exp_value(lsevalexp_binds)
            debug(False, 'lsevallthexp: ' + str(lsevallthexp))
            lsevalluminexp += get_eqn_elem_luminosity_exp_value(lsevalexp_binds)
            debug(False, 'lsevalluminexp: ' + str(lsevalluminexp))
            lsevalmassexp  += get_eqn_elem_mass_exp_value(lsevalexp_binds)
            debug(False, 'lsevalmassexp: ' + str(lsevalmassexp))
            lsevaltempexp  += get_eqn_elem_temperature_exp_value(lsevalexp_binds)
            debug(False, 'lsevaltempexp: ' + str(lsevaltempexp))
            lsevaltimeexp  += get_eqn_elem_time_exp_value(lsevalexp_binds)
            debug(False, 'lsevaltimeexp: ' + str(lsevaltimeexp))
            lsevaldlessexp += get_eqn_elem_dimensionless_exp_value(lsevalexp_binds)
            debug(False, 'lsevaldlessexp: ' + str(lsevaldlessexp))

        ls_eqn_eval = construct_eval_dv_from_exps(lsevalamtexp, lsevalcrtexp, lsevallthexp, lsevalluminexp, lsevalmassexp, lsevaltempexp, lsevaltimeexp, lsevaldlessexp)

    lseqnevallabel    = widgets.Label(layout = eval_vb_item_layout, value = ls_eqn_eval)
    ls_eqn_eval_items.append(lseqnevallabel)
    lsDABox.children  = ls_eqn_eval_items

    if (ls_eqn_eval == rs_eqn_eval):
        evalHTML.children = 'The equation is balanced'
    else:
        evalHTML.children = 'The equation is unbalanced'


# find_elem_tuple_from_option_label: Given a label, match it to the lapel in tuples and return the matching tuple
#
# ARGUMENTS: label  - the label for the tuple we are looking
#            tuples - the list of tuples to test against
# CALLED BY: update_ls_eqn_from_button
#            update_rs_eqn_from_button
# RETURN:    tuple  - the desired tuple

def find_elem_tuple_from_option_label(label, tuples):
    debug(False, 'In find_elem_tuple - label: ' + label)

    for tuple in tuples:
        tlabel  = tuple[1]
        tsymbol = tuple[2]
        qklsss  = tlabel + " - " + tsymbol
        debug(False, 'label: ' + label, 'tuple label: ' + qklsss)

#        if (label == qklsss):
        if (label.startswith(tlabel)):
            return tuple


# qk_value_for_eqn_elem: Given an equation element, return the value for the element
# ARGUMENTS: eqn_elem - An eqn element
# RETURN:    The value component of the eqn element
# CALLED BY: update_ls_eqn_from_button
#            update_rs_eqn_from_button

def qk_value_for_eqn_elem(eqn_elem):
    return eqn_elem[0]

#def qk_value_for_elem_symbol(eqn_elem_symbol, qks, qk_symbols, qk_labels, qk_values):
#    qk_label = ''
#    i        = 0
#    while (i < len(qk_symbols)):
#        qk_symbol = qk_symbols[i]
#        qk_label  = qk_labels[i]
#        if (str(qk_symbol) == eqn_elem_symbol and qk_label):
#            qk_value = qk_values[i]
#            return qk_value
#        i += 1


# on_change_lsqkb: Handles changes to the quantity kind button widget
#
# CALLED BY: Changes to ls_qk_button value
# CALLS:     update_ls_eqn_from_button

def on_change_lsqkb(change):
    debug(False, 'In on_change_lsqkb')
    update_ls_eqn_from_button()

ls_qk_button.on_click(on_change_lsqkb)


# update_rs_eqn_from_button: Performs model updates downstream of a right equation side button selection
#                            It updates the right side equation quantity list and gets the selected
#                            quantity kind's dimension vector and adds to the right side equation dimension
#                            vector list.
#
# ALGORITHM: (1) Append the selected quantity kind to the equation elements list
#            (2) Get the associated dimension vector for this quantity kind
#            (3) Construct the symbol/dimension vector string
#            (4) Append the string to the dimension vector list
#
# CALLED BY: on_change_rsqkb
# CALLS:     update_eqn_elem_content
#            eqn_elem_dv_get_request
#            get_rs_eqn_elem_dv_value
#            get_rs_eqn_elem_dv_label
#            construct_qk_symbols_dv_tuple
#            update_rs_eqn_dv_content
#            qk_value_for_qk_label
#            eqn_elem_dv_exps_get_request
#            get_eqn_elem_amount_exp_value
#            get_eqn_elem_current_exp_value
#            get_eqn_elem_length_exp_value
#            get_eqn_elem_luminosity_exp_value
#            get_eqn_elem_mass_exp_value
#            get_eqn_elem_temperature_exp_value
#            get_eqn_elem_time_exp_value
#            get_eqn_elem_dimensionless_exp_value
#            construct_eval_dv_from_exps

def update_rs_eqn_from_button(*args): # *args represent zero (case here) or more arguments.
    global rs_eqn_elems
    global rs_init_display
    global rs_dv_elems
    global rs_eqn_eval

    rsevalamtexp      = 0
    rsevalcrtexp      = 0
    rsevallthexp      = 0
    rsevalluminexp    = 0
    rsevalmassexp     = 0
    rsevaltempexp     = 0
    rsevaltimeexp     = 0
    rsevaldlessexp    = 0
    rs_eqn_eval_items = []

    debug(False, 'In update_rs_eqn_from_button - rs_qk_dd.index: ' + str(rs_qk_dd.index))
    debug(False, 'rs_eqn_elems BEFORE append:', rs_eqn_elems)

    selected_qk_label = rs_qk_dd.options[rs_qk_dd.index]
    selected_elem     = find_elem_tuple_from_option_label(selected_qk_label, rs_qks)
    debug(False, 'selected_elem:', selected_elem)

    if (rs_init_display == False):
        rs_eqn_elems.append(selected_elem)

    update_rs_eqn_elem_content(selected_elem)

    debug(False, 'length of rs_eqn_elems: ' + str(len(rs_eqn_elems)), 'rs_eqn_elems after append:', rs_eqn_elems)

    # Get the dimension vector for the selected quantity kind

    rseqndv_binds = eqn_elem_dv_get_request(rs_qkv_dd.options[rs_qk_dd.index]) # not returning the right value so changed the original array to the current options array
    rseqndvv      = get_eqn_elem_dv_value(rseqndv_binds)
    rseqndvl      = get_eqn_elem_dv_label(rseqndv_binds)

    debug(False, 'rseqndvl:', rseqndvl)

    # Construct the quantity kind, label, and symbol tuple

    rseqnqksdv = construct_qk_symbols_dv_tuple(rs_qk_dd.options[rs_qk_dd.index], rseqndvl, rs_qk_dd.options, rs_qks_dd.options)

    debug(False, 'rs_dv_elems BEFORE append:', rs_dv_elems, 'rseqnqksdv:', rseqnqksdv)

    if (rs_init_display == False):
        rs_dv_elems.append(rseqnqksdv)

    update_rs_eqn_dv_content(rseqnqksdv)

    debug(False, 'rs_dv_elems AFTER append:', rs_dv_elems)

    rs_init_display = False

    # Construct evaluation component content (not really written yet but will have to include these kinds of things)

    if (len(rs_eqn_elems) == 1):
        rs_eqn_eval = rseqnqksdv[1]
    else:
        for eqn_elem in rs_eqn_elems:
            debug(False, 'Construct evaluation component')
            debug(False, 'eqn_elem: ', eqn_elem, 'rs_qk_symbols:', rs_qk_symbols, 'rs_qk_values:', rs_qk_values)
            rs_qk_value     = qk_value_for_eqn_elem(eqn_elem)
#            rs_qk_value     = elem[0]
#            rs_qk_value     = qk_value_for_elem_symbol(eqn_elem, rs_qk_symbols, rs_qk_labels, rs_qk_values)
            debug(False, 'rs_qk_value: ', rs_qk_value)
            rsevalexp_binds = eqn_elem_dv_exps_get_request(rs_qk_value)
            debug(False, 'rsevalexp_binds:', rsevalexp_binds)
            rsevalamtexp   += get_eqn_elem_amount_exp_value(rsevalexp_binds)
            debug(False, 'rsevalamtexp: ' + str(rsevalamtexp))
            rsevalcrtexp   += get_eqn_elem_current_exp_value(rsevalexp_binds)
            debug(False, 'rsevalcrtexp: ' + str(rsevalcrtexp))
            rsevallthexp   += get_eqn_elem_length_exp_value(rsevalexp_binds)
            debug(False, 'rsevallthexp: ' + str(rsevallthexp))
            rsevalluminexp += get_eqn_elem_luminosity_exp_value(rsevalexp_binds)
            debug(False, 'rsevalluminexp: ' + str(rsevalluminexp))
            rsevalmassexp  += get_eqn_elem_mass_exp_value(rsevalexp_binds)
            debug(False, 'rsevalmassexp: ' + str(rsevalmassexp))
            rsevaltempexp  += get_eqn_elem_temperature_exp_value(rsevalexp_binds)
            debug(False, 'rsevaltempexp: ' + str(rsevaltempexp))
            rsevaltimeexp  += get_eqn_elem_time_exp_value(rsevalexp_binds)
            debug(False, 'rsevaltimeexp: ' + str(rsevaltimeexp))
            rsevaldlessexp += get_eqn_elem_dimensionless_exp_value(rsevalexp_binds)
            debug(False, 'rsevaldlessexp: ' + str(rsevaldlessexp))

        rs_eqn_eval = construct_eval_dv_from_exps(rsevalamtexp, rsevalcrtexp, rsevallthexp, rsevalluminexp, rsevalmassexp, rsevaltempexp, rsevaltimeexp, rsevaldlessexp)

    rseqnevallabel   = widgets.Label(layout = eval_vb_item_layout, value = rs_eqn_eval)
    rs_eqn_eval_items.append(rseqnevallabel)
    rsDABox.children = rs_eqn_eval_items

    if (ls_eqn_eval == rs_eqn_eval):
        evalHTML.children = 'The equation is balanced'
    else:
        evalHTML.children = 'The equation is unbalanced'


# on_change_rsqkb: Handles changes to the quantity kind button widget
#
# CALLED BY: Changes to ls_qk_button value
# CALLS:     update_ls_eqn_from_button

def on_change_rsqkb(change):
    update_rs_eqn_from_button()

rs_qk_button.on_click(on_change_rsqkb)


# eqn_elem_dv_get_request: Constructs and issues the equation element dimension vector SPARQL query, 
#                          converts to json, and parses the ressulting results and bindings values. The 
#                          bindings are returned.
#
# ARGUMENTS: qk_value    - the selected quanity kind from the qk_dd dropdown
# RETURN:    eqelembinds - the SPARQL query bindings array
# CALLED BY: update_eqn_from_button

def eqn_elem_dv_get_request(qk_value):
    debug(False, 'In eqn_elem_dv_get_request - qk_value', qk_value)
    eqn_elem_dv_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?qkdv ?qkdvl WHERE { <"
    eqn_elem_dv_query_end   = "> qudt:hasDimensionVector ?qkdv . ?qkdv rdfs:label ?qkdvl}"
    eqn_elem_dv_query       = eqn_elem_dv_query_start + qk_value + eqn_elem_dv_query_end
    debug(False, 'eqn_elem_dv_query:', eqn_elem_dv_query)
    eqn_elem_dv_response    = requests.get(eqn_elem_dv_query, headers = my_headers)
    eqndv_data              = eqn_elem_dv_response.json()
    eqndv_results           = eqndv_data['results']
    eqndv_binds             = eqndv_results['bindings']
    debug(False, 'eqndv_binds:', eqndv_binds)

    return eqndv_binds


# get_eqn_elem_dv_value: Parses the 'qkdv' value from the SPARQL query results.
#
# ARGUMENTS: obj               - a bindings array
# RETURN:    eqn_elem_dv_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_dv_value(obj):
    elem              = obj[0]
    value             = elem['qkdv']
    eqn_elem_dv_value = value['value']

    return eqn_elem_dv_value


# get_eqn_elem_dv_label: Parses the 'qkdvl' value from the SPARQL query results.
#
# ARGUMENTS: obj               - a bindings array
# RETURN:    eqn_elem_dv_label - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_dv_label(obj):
    elem              = obj[0]
    value             = elem['qkdvl']
    eqn_elem_dv_label = value['value']

    return eqn_elem_dv_label


# construct_qk_symbols_dv_tuple: Constructs a tuple for a QK's symbol and dimension vector.
#
# ARGUMENTS: qk    - The quantity kind 
#            dv    - The dimension vector
#            vlist - The list of values
#            slist - The list of symbols
# CALLED BY: update_ls_qk_options_from_input_text
# CALLS:     [python] startswith

def construct_qk_symbols_dv_tuple(qk, dv, vlist, slist):
    debug(False, 'In construct_qk_symbols_dv_tuple', qk, dv, vlist, slist)

    i = 0

    while i < len(vlist):
        item = vlist[i]

        if item.startswith(qk):
            selem = slist[i]

        i += 1

    return selem, dv


# update_ls_eqn_from_eqn_elem_button: When selected, remove the associated element from the current
#                                     equation and update other lists
#
# ALGORITHM: (1)
#
# CALLED BY: on_change_lseqnbtn
# CALLS:     d

def update_ls_eqn_from_eqn_elem_button():
    global ls_eqn_elems
    global ls_eqn_btns

    debug(False, 'In update_ls_eqn_from_eqn_elem_button')


# on_change_lseqnbtn: Handles changes to the equation element button widget
#
# CALLED BY: Changes to ls_eqn_btns value
# CALLS:     update_ls_eqn_from_eqn_elem_button

def on_change_lseqnbtn(change):
    debug(False, 'in on_change_lseqnbtn')
    update_ls_eqn_from_eqn_elem_button()


# update_rs_eqn_from_eqn_elem_button: When selected, remove the associated element from the current
#                                     equation and update other lists
#
# ALGORITHM: (1)
#
# CALLED BY: on_change_rseqnbtn
# CALLS:     d

def update_rs_eqn_from_eqn_elem_button():
    global rs_eqn_elems
    global rs_eqn_btns

    debug(False, 'In update_ls_eqn_from_eqn_elem_button')


# on_change_rseqnbtn: Handles changes to the equation element button widget
#
# CALLED BY: Changes to rs_eqn_btn value
# CALLS:     update_rs_eqn_from_eqn_elem_button

def on_change_rseqnbtn(change):
    update_rs_eqn_from_eqn_elem_button()


# eqn_elem_dv_exps_get_request: Constructs and issues the equation element dimension vector exponents
#                               SPARQL query, converts to json, and parses the ressulting results and bindings 
#                               values. The bindings are returned. This is used for evaluation.
#
# ALGORITHM: (1) Construct the equation elements dimension vector exponents query based on a selected quantity
#                kind's value
#            (2) Invoke the GET request with this query
#            (3) Convert the result into JSON
#            (4) Parse the BINDINGS value from the JSON
#            (5) Return the bindings list
#
# ARGUMENTS: qk_value        - the selected quantity kind from the qk_dd dropdown
# RETURN:    eqndvexps_binds - the SPARQL query bindings array for the dv exponents
# CALLED BY: update_ls_eqn_from_button
#            update_rs_eqn_from_button
# CALLS:     requests.get

def eqn_elem_dv_exps_get_request(qk_dv_value):
    debug(False, 'In eqn_elem_dv_exps_get_request - qk_dv_value', qk_dv_value)
    eqn_elem_dv_exps_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?qkdv ?qkdvas ?qkdvec ?qkdvl ?qkdvli ?qkdvm ?qkdvtt ?qkdvt ?qkdvd WHERE { <"
    eqn_elem_dv_exps_query_end   = "> qudt:hasDimensionVector ?qkdv . ?qkdv qudt:dimensionExponentForAmountOfSubstance ?qkdvas . ?qkdv qudt:dimensionExponentForElectricCurrent ?qkdvec . ?qkdv qudt:dimensionExponentForLength ?qkdvl . ?qkdv qudt:dimensionExponentForLuminousIntensity ?qkdvli . ?qkdv qudt:dimensionExponentForMass ?qkdvm . ?qkdv qudt:dimensionExponentForThermodynamicTemperature ?qkdvtt . ?qkdv qudt:dimensionExponentForTime ?qkdvt . ?qkdv qudt:dimensionlessExponent ?qkdvd .}"
    eqn_elem_dv_exps_query       = eqn_elem_dv_exps_query_start + qk_dv_value + eqn_elem_dv_exps_query_end
    debug(False, 'eqn_elem_dv_exps_query', eqn_elem_dv_exps_query)
    eqn_elem_dv_exps_response    = requests.get(eqn_elem_dv_exps_query, headers = my_headers)
    eqndvexps_data               = eqn_elem_dv_exps_response.json()
    eqndvexps_results            = eqndvexps_data['results']
    eqndvexps_binds              = eqndvexps_results['bindings']
    debug(False, 'eqndvexps_binds', eqndvexps_binds)

    return eqndvexps_binds


# get_eqn_elem_amount_exp_value: Parses the 'qkdvas' (amount) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                       - a bindings array
# RETURN:    eqn_elem_amount_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_amount_exp_value(obj):
    elem                      = obj[0]
    value                     = elem['qkdvas']
    eqn_elem_amount_exp_value = value['value']

    return int(eqn_elem_amount_exp_value)


# get_eqn_elem_current_exp_value: Parses the 'qkdvec' (current) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                        - a bindings array
# RETURN:    eqn_elem_current_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_current_exp_value(obj):
    elem                       = obj[0]
    value                      = elem['qkdvec']
    eqn_elem_current_exp_value = value['value']

    return int(eqn_elem_current_exp_value)


# get_eqn_elem_length_exp_value: Parses the 'qkdvl' (length) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                       - a bindings array
# RETURN:    eqn_elem_length_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_length_exp_value(obj):
    elem                      = obj[0]
    value                     = elem['qkdvl']
    eqn_elem_length_exp_value = value['value']

    return int(eqn_elem_length_exp_value)


# get_eqn_elem_luminosity_exp_value: Parses the 'qkdvli' (luminosity) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                           - a bindings array
# RETURN:    eqn_elem_luminosity_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_luminosity_exp_value(obj):
    elem                          = obj[0]
    value                         = elem['qkdvli']
    eqn_elem_luminosity_exp_value = value['value']

    return int(eqn_elem_luminosity_exp_value)


# get_eqn_elem_mass_exp_value: Parses the 'qkdvm' (mass) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                     - a bindings array
# RETURN:    eqn_elem_mass_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_mass_exp_value(obj):
    elem                    = obj[0]
    value                   = elem['qkdvm']
    eqn_elem_mass_exp_value = value['value']

    return int(eqn_elem_mass_exp_value)


# get_eqn_elem_temperature_exp_value: Parses the 'qkdvtt' (temperature) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                            - a bindings array
# RETURN:    eqn_elem_temperature_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_temperature_exp_value(obj):
    elem                           = obj[0]
    value                          = elem['qkdvtt']
    eqn_elem_temperature_exp_value = value['value']

    return int(eqn_elem_temperature_exp_value)


# get_eqn_elem_time_exp_value: Parses the 'qkdvt' (time) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                     - a bindings array
# RETURN:    eqn_elem_time_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_time_exp_value(obj):
    elem                    = obj[0]
    value                   = elem['qkdvt']
    eqn_elem_time_exp_value = value['value']

    return int(eqn_elem_time_exp_value)


# get_eqn_elem_dimensionless_exp_value: Parses the 'qkdvd' (dimensionless) value from the 'exponents' SPARQL query results.
#
# ARGUMENTS: obj                              - a bindings array
# RETURN:    eqn_elem_dimensionless_exp_value - the quantity kind SPARQL query labels array
# CALLED BY: statement below

def get_eqn_elem_dimensionless_exp_value(obj):
    elem                             = obj[0]
    value                            = elem['qkdvd']
    eqn_elem_dimensionless_exp_value = value['value']

    return int(eqn_elem_dimensionless_exp_value)


# construct_eval_dv_from_exps: Given the 8 aggregrated exponents, create a dimension vector string
#
# ARGUMENTS: amtexp   - aggregated amount of substance exponent
#            crtexp   - aggregated electrical current exponent
#            lthexp   - aggregated length exponent
#            luminexp - aggregated luminous intensity exponent
#            massexp  - aggregated mass exponent
#            tempexp  - aggregated temperature exponent
#            timeexp  - aggregated time exponent
#            dlessexp - aggregated dimensionless exponent

def construct_eval_dv_from_exps(amtexp, crtexp, lthexp, luminexp, massexp, tempexp, timeexp, dlessexp):
    dv_str = 'A' + str(amtexp) + 'E' + str(crtexp) + 'L' + str(lthexp) + 'I' + str(luminexp) + 'M' + str(massexp) + 'H' + str(tempexp) + 'T' +str(timeexp) + 'D' + str(dlessexp)
    return dv_str


# update_ls_eqn_elem_content: When the select quantity kind button is pressed the content of the equation element
#                             button array will be updated. That is, a new button will be added to the button
#                             array. The button widgets have already been defined.
#
# ARGUMENTS: cur_elem is the current qk dropdown element (a label), which will override the first element if the
#            first button selection
# ALGORITHM: (1) Iterate through all of the ls_eqn_elems items
#                (a) If this is from the first button press (i.e., ls_init_display is True) and it is the first
#                    element in the list,
#                    (i) Reassign the element the value from the dropdown
#                (b) Otherwise (ls_iinit_display is False or it isn't the first element of the list)
#                    (i) Create a new button from the element item
#                   (ii) Append the new button to the button list
#            (2) Assign the carousel children the value of the new list
# CALLED BY: update_ls_eqn_from_button

def update_ls_eqn_elem_content(cur_elem):
    global ls_eqn_elems
    global ls_eqn_btns

    debug(False, 'In update_ls_eqn_elem_content')
    debug(False, 'BEFORE loop: length of ls_eqn_elems: ' + str(len(ls_eqn_elems)), 'ls_eqn_elems:', ls_eqn_elems)
    debug(False, 'BEFORE loop: length of ls_eqn_btns: ' + str(len(ls_eqn_btns)), 'ls_eqn_btns:', ls_eqn_btns)

    if (ls_init_display == True):
        debug(False, 'Initial item in ls_eqn_elems, overwrite button description')
        ls_eqn_elems[0]    = cur_elem
        eqnbtn             = ls_eqn_btns[0]
        eqnbtn.description = cur_elem[2]
    else:
        debug(False, 'Not the initial item in ls_eqn_elems, or not the first time through; create a new button')
        lseqnbtn = widgets.Button(layout = eqn_hb_item_layout, description = cur_elem[2])
        lseqnbtn.on_click(on_change_lseqnbtn)
        ls_eqn_btns.append(lseqnbtn)
        # recreate the carousel with the ls_eqn_btns list

    ls_elem_carousel.children = ls_eqn_btns

    debug(False, 'AFTER loop: length of ls_eqn_elems: ' + str(len(ls_eqn_elems)), 'ls_eqn_elems:', ls_eqn_elems)
    debug(False, 'AFTER loop: length of ls_eqn_btns: ' + str(len(ls_eqn_btns)), 'ls_eqn_btns:', ls_eqn_btns)


# update_rs_eqn_elem_content: When the select quantity kind button is pressed the content of the equation element
#                             button array will be updated. The button widgets have already been defined.
#
# ARGUMENTS: cur_elem is the current qk dropdown element, which will override the first element if the first
#            button selection
# CALLED BY: update_rs_eqn_from_button

def update_rs_eqn_elem_content(cur_elem):
    global rs_eqn_elems
    global rs_eqn_btns

    debug(False, 'In update_rs_eqn_elem_content: length of rs_eqn_elems: ' + str(len(ls_eqn_elems)), 'rs_eqn_elems:', rs_eqn_elems, 'length of rs_eqn_btns: ' + str(len(ls_eqn_btns)), 'rs_eqn_btns:', rs_eqn_btns)

    if (rs_init_display == True):
        debug(False, 'This is the initial item in rs_eqn_elems, overwrite button description')
        rs_eqn_elems[0]    = cur_elem
        eqnbtn             = rs_eqn_btns[0]
        eqnbtn.description = cur_elem[2]
    else:
        debug(False, 'This is not the initial item in rs_eqn_elems, create a new button')
        rseqnbtn = widgets.Button(layout = eqn_hb_item_layout, description = cur_elem[2])
        rseqnbtn.on_click(on_change_rseqnbtn)
        rs_eqn_btns.append(rseqnbtn)

    rs_elem_carousel.children = rs_eqn_btns


# update_ls_eqn_dv_content: When the select quantity kind button is pressed the content of the dimension vector
#                           symbol label array and dimension vector label array will be updated.
#
# ARGUMENTS: cur_elem is the current dv element, which will override the first element if the first
#            button selection
# CALLED BY: update_ls_eqn_from_button

def update_ls_eqn_dv_content(cur_elem):
    global ls_dv_elems
    global ls_dv_vb_slabels
    global ls_dv_vb_dvlabels

    debug(False, 'In update_ls_eqn_dv_content')
    debug(False, 'BEFORE loop: length of ls_dv_elems: ' + str(len(ls_dv_elems)), 'ls_dv_elems:', ls_dv_elems)
    debug(False, 'length of ls_dv_vb_slabels: ' + str(len(ls_dv_vb_slabels)), 'ls_dv_vb_slabels:', ls_dv_vb_slabels)
    debug(False, 'length of ls_dv_vb_dvlabels: ' + str(len(ls_dv_vb_dvlabels)), 'ls_dv_vb_dvlabels:', ls_dv_vb_dvlabels)

    if (ls_init_display == True):
        debug(False, 'This is the initial element, overwrite the slabel value and dvlabel value')
        ls_dv_elems[0]  = cur_elem
        slabitem        = ls_dv_vb_slabels[0]
        slabitem.value  = cur_elem[0]
        dvlabitem       = ls_dv_vb_dvlabels[0]
        dvlabitem.value = cur_elem[1]
    else:
        debug(False, 'This is not the initial element create a new elem, slabel, and dvlabel')
        lsslabel        = widgets.Label(layout = dv_vb_item_layout, value = cur_elem[0])
        ls_dv_vb_slabels.append(lsslabel)
        lsdvlabel       = widgets.Label(layout = dv_vb_item_layout, value = cur_elem[1])
        ls_dv_vb_dvlabels.append(lsdvlabel)

    ls_dv_slab_VBox.children  = ls_dv_vb_slabels
    ls_dv_dvlab_VBox.children = ls_dv_vb_dvlabels

    debug(False, 'AFTER loop: length of ls_dv_elems: ' + str(len(ls_dv_elems)), 'ls_dv_elems:', ls_dv_elems)
    debug(False, 'length of ls_dv_vb_slabels: ' + str(len(ls_dv_vb_slabels)), 'ls_dv_vb_slabels:', ls_dv_vb_slabels)
    debug(False, 'length of ls_dv_vb_dvlabels: ' + str(len(ls_dv_vb_dvlabels)), 'ls_dv_vb_dvlabels:', ls_dv_vb_dvlabels)


# update_rs_eqn_dv_content: When the select quantity kind button is pressed the content of the dimension vector
#                           symbol label array and dimension vector label array will be updated. The label widgets
#                           have already been defined.
#
# ARGUMENTS: cur_elem is the current dv element, which will override the first element if the first
#            button selection
# CALLED BY: update_rs_eqn_from_button

def update_rs_eqn_dv_content(cur_elem):
    global rs_dv_elems
    global rs_dv_vb_slabels
    global rs_dv_vb_dvlabels

    debug(False, 'In update_ls_eqn_dv_content')
    debug(False, 'BEFORE loop: length of rs_dv_elems: ' + str(len(rs_dv_elems)), 'rs_dv_elems:', rs_dv_elems)
    debug(False, 'length of rs_dv_vb_slabels: ' + str(len(rs_dv_vb_slabels)), 'rs_dv_vb_slabels:', rs_dv_vb_slabels)
    debug(False, 'length of rs_dv_vb_dvlabels: ' + str(len(rs_dv_vb_dvlabels)), 'rs_dv_vb_dvlabels:', rs_dv_vb_dvlabels)

    if (rs_init_display == True):
        debug(False, 'This is the initial element, overwrite the slabel value and dvlabel value')
        rs_dv_elems[0]  = cur_elem
        slabitem        = rs_dv_vb_slabels[0]
        slabitem.value  = cur_elem[0]
        dvlabitem       = rs_dv_vb_dvlabels[0]
        dvlabitem.value = cur_elem[1]
    else:
        debug(False, 'This is not the initial element, create a new slabel and dvlabel')
        rsslabel  = widgets.Label(layout = dv_vb_item_layout, value = cur_elem[0])
        rs_dv_vb_slabels.append(rsslabel)
        rsdvlabel = widgets.Label(layout = dv_vb_item_layout, value = cur_elem[1])
        rs_dv_vb_dvlabels.append(rsdvlabel)

    rs_dv_slab_VBox.children  = rs_dv_vb_slabels
    rs_dv_dvlab_VBox.children = rs_dv_vb_dvlabels

    debug(False, 'AFTER loop: length of rs_dv_elems: ' + str(len(rs_dv_elems)), 'rs_dv_elems:', rs_dv_elems)
    debug(False, 'length of rs_dv_vb_slabels: ' + str(len(rs_dv_vb_slabels)), 'rs_dv_vb_slabels:', rs_dv_vb_slabels)
    debug(False, 'length of rs_dv_vb_dvlabels: ' + str(len(rs_dv_vb_dvlabels)), 'rs_dv_vb_dvlabels:', rs_dv_vb_dvlabels)


# update_ls_eqn_eval_content: When the select quantity kind button is pressed the content of the evaluation element
#                             exponent array will be updated and the evaluation performed.
#
# ALGORITHM: (1) Itereate over the dimension vector elements
#                (a) Get a dimension vector element
#                (b) If the length of the dimension vector elements list is 1 (we haven't hit the button twice)
#                    (i) Get the dv label
#                   (ii) Assign the label value from the dimension vector element's label
#                (c) Otherwise
#                    (i) Create a new label whose value is the dimension vector element's label
#                   (ii) Append the new label to the dimension vectore labels array
# CALLED BY: update_ls_eqn_from_button

def update_ls_eqn_eval_content():
    debug(False, 'In update_ls_eqn_eval_content')

    i = 0

    while i < len(ls_dv_elems):
        debug(False, 'ls eqn eval loop index:', i, 'dvitem:', ls_dv_elems[i], 'eqnbtn:', ls_dv_vb_dvlabels[i])
        dvitem = ls_dv_elems[i]

        if (len(ls_dv_elems) == 1):
            evaldvlabel = ls_dv_vb_dvlabels[i]
            evaldvlabel.value = dvitem[0]
        else:
            lsdvlabel = widgets.Label(layout = eval_vb_item_layout, value = dvitem[0])
            ls_dv_vb_dvlabels.append(lsdvlabel)

        i += 1


# update_rs_eqn_eval_content: When the select quantity kind button is pressed the content of the evaluation element
#                             exponent array will be updated and the evaluation performed.
#
# ALGORITHM: (1) Itereate over the dimension vector elements
#                (a) Get a dimension vector element
#                (b) If the length of the dimension vector elements list is 1 (we haven't hit the button twice)
#                    (i) Get the dv label
#                   (ii) Assign the label value from the dimension vector element's label
#                (c) Otherwise
#                    (i) Create a new label whose value is the dimension vector element's label
#                   (ii) Append the new label to the dimension vectore labels array
# CALLED BY: update_rs_eqn_from_button

def update_rs_eqn_eval_content():
    debug(False, 'In update_rs_eqn_eval_content')

    i = 0

    while i < len(rs_dv_elems):
        dvitem             = rs_dv_elems[i]
        eqnbtn             = rseqhb_items[i]
        debug(False, 'rs eqn eval loop index:', i, 'item:', dvitem, 'btn', eqnbtn)
        eqnbtn.description = dvitem[0]
        i += 1


###########################################################################################
# Construct the the baseline app display
###########################################################################################

# Define the quantity kind elements (text, dropdown, and button), and box, and display them

qk_vb_layout = widgets.Layout(border = 'solid 2px', margin = '7px', padding = '3px')
lsQKVBox     = widgets.VBox([ls_qk_text, ls_qk_dd, ls_qk_button], layout = qk_vb_layout)
rsQKVBox     = widgets.VBox([rs_qk_text, rs_qk_dd, rs_qk_button], layout = qk_vb_layout)

# Define and initialize the equation horizontal box elements carousels and display them

eqn_hb_item_layout = widgets.Layout(border = 'solid 1px', max_width = '40px', color = 'gray')
ls_eqn_elems.append('F')
rs_eqn_elems.append('M')
lseqnbtn           = widgets.Button(layout = eqn_hb_item_layout, description = 'F')
rseqnbtn           = widgets.Button(layout = eqn_hb_item_layout, description = 'M')
ls_eqn_btns.append(lseqnbtn)
lseqnbtn.on_click(on_change_rseqnbtn)
rs_eqn_btns.append(rseqnbtn)
rseqnbtn.on_click(on_change_rseqnbtn)
eqn_hb_layout      = widgets.Layout(overflow = 'scroll hidden', border = '1px solid green', flex_flow = 'row', display = 'flex')
ls_elem_carousel   = widgets.Box(children = ls_eqn_btns) #, layout = lseqhb_layout)
ls_eqn_elems_HBox  = widgets.HBox([ls_elem_carousel], layout = eqn_hb_layout)
rs_elem_carousel   = widgets.Box(children = rs_eqn_btns) #, layout = rseqhb_layout)
rs_eqn_elems_HBox  = widgets.HBox([rs_elem_carousel], layout = eqn_hb_layout)

# Define the dimension vector vertical box elements and display them

dv_vb_item_layout = widgets.Layout(border = 'solid 0px', min_height = '20px')
dv_vb_layout      = widgets.Layout(min_width = '30px')

lsslabel          = widgets.Label(layout = dv_vb_item_layout, value = 'F')
ls_dv_vb_slabels.append(lsslabel)
ls_dv_slab_VBox   = widgets.VBox(ls_dv_vb_slabels, layout = dv_vb_layout)
lsdvlabel         = widgets.Label(layout = dv_vb_item_layout, value = 'A0E0L1I0M1H0T-2D0')
ls_dv_vb_dvlabels.append(lsdvlabel)
ls_dv_elems       = [('F', 'A0E0L1I0M1H0T-2D0')]
ls_dv_dvlab_VBox  = widgets.VBox(ls_dv_vb_dvlabels)
ls_dv_vb_items    = [ls_dv_slab_VBox, ls_dv_dvlab_VBox]
ls_dv_elems_HBox  = widgets.HBox(ls_dv_vb_items)

rsslabel          = widgets.Label(layout = dv_vb_item_layout, value = 'm')
rs_dv_vb_slabels.append(rsslabel)
rs_dv_slab_VBox   = widgets.VBox(rs_dv_vb_slabels, layout = dv_vb_layout)
rsdvlabel         = widgets.Label(layout = dv_vb_item_layout, value = 'A0E0L0I0M1H0T0D0')
rs_dv_vb_dvlabels.append(rsdvlabel)
rs_dv_elems       = [('m', 'A0E0L0I0M1H0T0D0')]
rs_dv_dvlab_VBox  = widgets.VBox(rs_dv_vb_dvlabels)
rs_dv_vb_items    = [rs_dv_slab_VBox, rs_dv_dvlab_VBox]
rs_dv_elems_HBox  = widgets.HBox(rs_dv_vb_items)

# Define the equation and dimension vbox display

eqn_dv_vb_layout = widgets.Layout(border = 'solid 2px', margin = '7px', padding = '3px')
lsDVVBox         = widgets.VBox([ls_eqn_elems_HBox, ls_dv_elems_HBox], layout = eqn_dv_vb_layout) # ls_eqesHBox
rsDVVBox         = widgets.VBox([rs_eqn_elems_HBox, rs_dv_elems_HBox], layout = eqn_dv_vb_layout) # ls_eqesHBox

# Define the evaluation box elements and display them

# Define the evaluation vertical box elements and display them

eval_vb_item_layout = widgets.Layout(border = 'solid 0px', min_height = '20px')
eval_vb_layout      = widgets.Layout(min_width = '30px')

ls_eval_slabels    = []
lsevslabel         = widgets.Label(layout = eval_vb_item_layout, value = 'A0E0L1I0M1H0T-2D0')
ls_eval_slabels.append(lsevslabel)
ls_eval_slab_VBox  = widgets.VBox(ls_eval_slabels, layout = eval_vb_layout)
ls_eval_dvlabels   = []
lsevdvlabel        = widgets.Label(layout = eval_vb_item_layout, value = 'A0E0L1I0M1H0T-2D0')
ls_eval_dvlabels.append(lsevdvlabel)
ls_eval_dvlab_VBox = widgets.VBox(ls_eval_dvlabels)
ls_eval_vb_items   = [ls_eval_slab_VBox, ls_eval_dvlab_VBox]
ls_eval_dv_HBox    = widgets.HBox(ls_dv_vb_items)

rs_eval_vb_slabels = []
rsevslabel         = widgets.Label(layout = eval_vb_item_layout, value = 'A0E0L0I0M1H0T0D0')
rs_eval_vb_slabels.append(rsevslabel)
rsdvsvb            = widgets.VBox(rs_dv_vb_slabels, layout = eval_vb_layout)
rs_dv_vb_dvlabels  = []
rsdvlabel          = widgets.Label(layout = eval_vb_item_layout, value = 'A0E0L0I0M1H0T0D0')
rs_dv_vb_dvlabels.append(rsdvlabel)
rsdvdvvb           = widgets.VBox(rs_dv_vb_dvlabels)
rs_dv_vb_items     = [rsdvsvb, rsdvdvvb]
rs_dvesHBox        = widgets.HBox(rs_dv_vb_items)

# aggregate dv exps in eval:

eval_layout    = widgets.Layout(border = 'solid 2px', margin = '7px', padding = '3px')

lseqnevallabel = widgets.Label(layout = eval_vb_item_layout, value = 'A0E0L1I0M1H0T-2D0')
rseqnevallabel = widgets.Label(layout = eval_vb_item_layout, value = 'A0E0L0I0M1H0T0D0')
lsDABox        = widgets.Box([lseqnevallabel], layout = eval_layout)
rsDABox        = widgets.Box([rseqnevallabel], layout = eval_layout)

# Display the complete application

#app_grid_items  = [lseqLabel, rseqLabel, lsQKVbox, rsQKVBox, lsDVVBox, lsDVVBox, lsDABox, rsDABox]
app_grid_items  = [lseqnLabel, rseqnLabel, lsQKVBox, rsQKVBox, lsDVVBox, rsDVVBox, lsDABox, rsDABox]
app_layout      = widgets.Layout(grid_template_columns = "repeat(2,325px)", border = 'solid 2px', margin = '2px', padding = '10px')
dyn_box         = widgets.GridBox(app_grid_items, layout = app_layout)
title_layout    = widgets.Layout(margin = '20px', padding = '3px')
app_box         = widgets.VBox([titleBox, dyn_box, evalBox], layout = title_layout)
display(app_box)



IndexError: list index out of range